In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import folium
import cv2
import serial
import tkinter
# Plot values in opencv program
from Realtime_Plotter import Realtime_Plotter


In [2]:
class PWM_GUI:
    times = []
    throttles = []
    steers = []
    statuses = []
    controls = []
    
    max_vals = -1
    
    def __init__(self, max_vals):
        self.max_vals = max_vals
        self.steering_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=1000, max_val=2000, label="Steering")
        self.throttle_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=1000, max_val=2000, label="Throttle")
        self.df = pd.DataFrame(columns=['time', 'status', 'steer', 'throttle', 'control'])

    def ingest_line(self, line):
            # [0.01695728302001953, '1', '-1', '1502', '1497', '$']
            # print("pwm line: ", line)
            # print("time: ", line[0])
            line[1] = float(line[1])
            # print("status: ", line[1])
            line[2] = float(line[2])
            # print("control: ", line[2])
            line[3] = float(line[3])
            # print("steer: ", line[3])
            line[4] = float(line[4])
            # print("throttle: ", line[4])            
            
            # print("line: ", line)
            self.df.loc[len(self.df)] = line[:-1]
            self.times.append(line[0])
            self.statuses.append(line[1])
            self.controls.append(line[2])
            self.steers.append(line[3])
            self.throttles.append(line[4])

            
            while len(self.steers) > self.max_vals:
                self.steers.pop(0)
            while len(self.throttles) > self.max_vals:
                self.throttles.pop(0)
            while len(self.statuses) > self.max_vals:
                self.statuses.pop(0)
            while len(self.times) > self.max_vals:
                self.times.pop(0)
            
            
            self.steering_plotter.build_plot(line[3])
            self.throttle_plotter.build_plot(line[4])
            
            disp = np.hstack((np.zeros((400,50,3)), self.steering_plotter.plot, np.zeros((400,50,3)), self.throttle_plotter.plot, np.zeros((400,50,3))))
            cv2.imshow("Steering - Throttle", disp)
            key = cv2.waitKey(10)
            
            if key == 113:
                return False
            else:
                return True

In [3]:
class GPS_GUI:
    lats = []
    lons = []
    alts = []
    speeds = []
    times = []
    
    max_vals = -1
    
    def __init__(self, max_vals):
        self.max_vals = max_vals
        self.speed_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=-30.0, max_val=1.0, label="Speed")
        self.heading_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=-10.0, max_val=370.0, label="Heading")
        self.df = pd.DataFrame(columns=['time', 'lat', 'lon', 'alt', 'speed', 'heading', 'fusion_calibration'])

    def ingest_line(self, line):
            # print("line: ", line)
            # ['G', '394265191', '-1198031066', '1466181', '18', '0', '0', '$']
            line[1] = float(line[1]) / 10**7
            # print("lat: ", line[1])
            line[2] = float(line[2]) / 10**7
            # print("lon: ", line[2])
            line[3] = float(line[3]) / 10**3
            # print("alt: ", line[3])
            line[4] = float(line[4]) / 10**3
            # print("speed: ", line[4])
            line[5] = float(line[5]) / 10**5
            # print("heading: ", line[5])
            # print("cal status: ", line[6])
            self.df.loc[len(self.df)] = line[:-1]
            
            self.lats.append(line[1])
            self.lons.append(line[2])
            self.alts.append(line[3])
            self.speeds.append(line[4])
            self.times.append(line[5])
            
            while len(self.lats) > self.max_vals:
                self.lats.pop(0)
            while len(self.lons) > self.max_vals:
                self.lons.pop(0)
            while len(self.alts) > self.max_vals:
                self.alts.pop(0)
            while len(self.speeds) > self.max_vals:
                self.speeds.pop(0)
            while len(self.times) > self.max_vals:
                self.times.pop(0)
            
            self.speed_plotter.build_plot(line[4])
            self.heading_plotter.build_plot(line[5])
            
            disp = np.hstack((np.zeros((400,50,3)), self.speed_plotter.plot, np.zeros((400,50,3)), self.heading_plotter.plot, np.zeros((400,50,3))))
            cv2.imshow("Speed - Heading", disp)
            key = cv2.waitKey(10)
            
            if key == 113:
                return False
            else:
                return True 
        

In [6]:
class RF_Interface:
    
    pwm_gui = PWM_GUI(max_vals=20)
    gps_gui = GPS_GUI(max_vals=20)
    
    def __init__(self, s_port='com20', baud=9600):
        self.ser = serial.Serial()
        ser_open = False
        attempts = 0

        #attempt to connect
        while not ser_open and attempts < 5:
            try:
                self.ser = serial.Serial(s_port, baud, timeout=None)
                ser_open = True
            except:
                print("Failed to open serial port: ", attempts)
                
            if not ser_open:
                time.sleep(0.5)
                attempts += 1
                
    def check_and_log(self, c_time):
        if self.ser.in_waiting:
            bytes = self.ser.readline()
            # print("bytes: ", bytes)
            try:
                decoded_strings = str(bytes[0:len(bytes)-2].decode("utf-8")).split(',')
                # print(decoded_strings)
                if decoded_strings[0] == 'G':
                    decoded_strings[0] = c_time
                    print("GPS decoded strings: ", decoded_strings)
                    self.gps_gui.ingest_line(decoded_strings)
                elif decoded_strings[0] == 'P':
                    decoded_strings[0] = c_time
                    print("PWM decoded strings: ", decoded_strings)
                    self.pwm_gui.ingest_line(decoded_strings)
                elif decoded_strings[0] == 'E':
                    print("ERR: ", decoded_strings)
                else:
                    print("Error: BAD STRING: ",  decoded_strings)
            except:
                print("RF didn't log correctly: ", bytes)
                
    def close_and_save(self):
        self.ser.close()
        
    def flush_logs(self):
        self.ser.flushInput()
        self.df = pd.DataFrame(columns=self.cols)
        

rf = RF_Interface()

start = time.time()
max_time = 15.0
c_time = time.time() - start
while c_time < max_time:
    c_time = time.time() - start
    rf.check_and_log(c_time)

rf.ser.close()
cv2.destroyAllWindows()
# rf.gps_gui.df


Error: BAD STRING:  ['85', '1099', '1107', '1793', '$']
PWM:  [0.030903100967407227, '1', '301405', '1100', '1108', '1793', '$']
RF didn't log correctly:  b'P,1,301405,1100,1108,1793,$\r\n'
GPS:  [0.06298398971557617, '301427', '394391358', '-1197850755', '1366760', '52', '7458072', '0', '$']
RF didn't log correctly:  b'G,301427,394391358,-1197850755,1366760,52,7458072,0,$\r\n'
PWM:  [0.11105990409851074, '1', '301427', '1100', '1109', '1791', '$']
RF didn't log correctly:  b'P,1,301427,1100,1109,1791,$\r\n'
PWM:  [0.14298295974731445, '1', '301525', '1100', '1111', '1789', '$']
RF didn't log correctly:  b'P,1,301525,1100,1111,1789,$\r\n'
PWM:  [0.1750473976135254, '1', '301565', '1100', '1111', '1790', '$']
RF didn't log correctly:  b'P,1,301565,1100,1111,1790,$\r\n'
PWM:  [0.20685458183288574, '1', '301584', '1100', '1112', '1789', '$']
RF didn't log correctly:  b'P,1,301584,1100,1112,1789,$\r\n'
GPS:  [0.23896098136901855, '301605', '394391359', '-1197850754', '1366744', '11', '7458

In [5]:
# https://python-visualization.github.io/folium/quickstart.html
m = folium.Map(location=[rf.gps_gui.df.lat[0], rf.gps_gui.df.lon[0]], zoom_start=20)
trail_coordinates = []
for i in range(len(rf.gps_gui.df)):
    trail_coordinates.append((rf.gps_gui.df.lat[i], rf.gps_gui.df.lon[i]))


folium.PolyLine(trail_coordinates, tooltip="Coast").add_to(m)
folium.Circle(
    radius=5,
    location=[rf.gps_gui.df.lat[0], rf.gps_gui.df.lon[0]],
    popup="Start",
    color="crimson",
    fill=False).add_to(m)

folium.Circle(
    radius=1,        
    location=[rf.gps_gui.df.lat[0], rf.gps_gui.df.lon[0]],
    popup="Start",
    color="crimson",
    fill=True).add_to(m)

display(m)

IndexError: index 0 is out of bounds for axis 0 with size 0